# Build Simple Tool-Use AI Agents in LangGraph

Here we will extend the capability of the previously built Augmented LLM with feedback from the tool execution back to the LLM node to process it and generate human-like answers to user queries.

### Tool-based Agentic AI System

- Dynamic Decision-Making: LLM determines whether to directly respond or invoke a tool based on the query context.
- Seamless Tool Integration: External tools are integrated to handle specific tasks, such as real-time web queries or computations.
- Workflow Flexibility: Conditional routing ensures efficient task delegation:
  - Tool Required: Routes to tool execution.
  - No Tool Required: Ends the workflow with an LLM response.
- Feedback Loop: Incorporates a feedback loop to improve responses by combining LLM insights and tool outputs to further improve responses or call more tools if needed

![](https://i.imgur.com/DHxiOLl.png)


In [0]:
# !pip install -qqqq langchain==0.3.14
# !pip install -qqqq langchain-openai==0.3.0
# !pip install -qqqq langchain-community==0.3.14
# !pip install -qqqq langgraph==0.2.64

In [0]:
# %pip install -U -qqqq mlflow databricks-langchain pydantic databricks-agents unitycatalog-langchain[databricks]
# dbutils.library.restartPython()

In [0]:
# from getpass import getpass
# from typing import Annotated
# from typing_extensions import TypedDict
# from langgraph.graph.message import add_messages
# from databricks_langchain.uc_ai import (
#     DatabricksFunctionClient,
#     UCFunctionToolkit,
#     set_uc_function_client,
# )
# from langchain_openai import ChatOpenAI
# from langchain_core.tools import tool
# from databricks_langchain import ChatDatabricks

# from langgraph.graph import StateGraph, START, END
# from typing import Annotated, Literal

### Install Required Libraries

In [0]:
%pip install -U -qqqq mlflow langchain langgraph==0.3.4 databricks-langchain pydantic databricks-agents unitycatalog-langchain[databricks]
dbutils.library.restartPython()

### Import required libraries

In [0]:
from getpass import getpass
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from databricks_langchain.uc_ai import (
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from databricks_langchain import ChatDatabricks

from langgraph.graph import StateGraph, START, END
from typing import Annotated, Literal,Any, Optional, Sequence, Union
from langchain_core.language_models import LanguageModelLike
from langchain_core.tools import BaseTool
from langgraph.prebuilt.tool_node import ToolNode,tools_condition
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langgraph.graph.graph import CompiledGraph
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
from mlflow.models import ModelConfig

### Augment the LLM with tools (Testing purpose)

Here we define our custom search tool and then bind it to the LLM to augment the LLM

In [0]:
uc_client = DatabricksFunctionClient()
set_uc_function_client(uc_client)

LLM_ENDPOINT = "databricks-meta-llama-3-3-70b-instruct"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT)

catalog = "agentic_ai"
schema = "databricks"

uc_tool_names = [f"{catalog}.{schema}.search_web"]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools=[*uc_toolkit.tools]
llm_with_tools = llm.bind_tools(tools=tools)

In [0]:
llm_with_tools.invoke('what is the latest news on nvidia')

## Create Agent Not Compatible with MLFlow

### State

First, define the [State](https://langchain-ai.github.io/langgraph/concepts/low_level/#state) of the graph.

The State schema serves as the input schema for all Nodes and Edges in the graph.

Let's use the `TypedDict` class from python's `typing` module as our schema, which provides type hints for the keys.

In [0]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

### Create Tool Calling Agent

In [0]:
def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: list
) -> CompiledGraph:
    llm_with_tools = llm.bind_tools(tools=tools)

    # Augmented LLM with Tools Node function
    def tool_calling_llm(state: State) -> State:
        """Execute tools based on tool calls in the last message"""
        current_state = state["messages"]
        return {"messages": [llm_with_tools.invoke(current_state)]}
    
    # Build the graph
    builder = StateGraph(State)
    builder.add_node("tool_calling_llm", tool_calling_llm)
    builder.add_node("tools", ToolNode(tools=tools))
    builder.add_edge(START, "tool_calling_llm")

    # Conditional Edge
    builder.add_conditional_edges(
        "tool_calling_llm",
        # If the latest message (result) from LLM is a tool call -> tools_condition routes to tools
        # If the latest message (result) from LLM is a not a tool call -> tools_condition routes to END
        tools_condition,
        ["tools", END]
    )
    builder.add_edge("tools", "tool_calling_llm") # this is the key feedback loop
    builder.add_edge("tools", END)
    agent = builder.compile()

    return agent

catalog = "agentic_ai"
schema = "databricks"

# TODO: Replace with your model serving endpoint
LLM_ENDPOINT = "databricks-meta-llama-3-3-70b-instruct"

LLM_ENDPOINT = "databricks-meta-llama-3-3-70b-instruct"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT)

uc_client = DatabricksFunctionClient()
set_uc_function_client(uc_client)



uc_tool_names = [f"{catalog}.{schema}.search_web"]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools=[*uc_toolkit.tools]
agent = create_tool_calling_agent(llm, tools)
agent

### Call the Agent

In [0]:
user_input = "Explain AI in 2 bullets"
for event in agent.stream({"messages": user_input},
                          stream_mode='values'):
    event['messages'][-1].pretty_print()

In [0]:
user_input = "What is the latest news on OpenAI product releases. Provide the result in bullet points"
for event in agent.stream({"messages": user_input},
                          stream_mode='values'):
    event['messages'][-1].pretty_print()

In [0]:
event['messages'][-1]

In [0]:
from IPython.display import display, Markdown

display(Markdown(event['messages'][-1].content))

## Create Agent  Compatible with MLFlow

In [0]:
from getpass import getpass
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from databricks_langchain.uc_ai import (
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)

import mlflow
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from databricks_langchain import ChatDatabricks

from langgraph.graph import StateGraph, START, END
from typing import Annotated, Literal,Any, Optional, Sequence, Union
from langchain_core.language_models import LanguageModelLike
from langchain_core.tools import BaseTool
from langgraph.prebuilt.tool_node import ToolNode,tools_condition
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langgraph.graph.graph import CompiledGraph
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
from mlflow.models import ModelConfig

class State(TypedDict):
    messages: Annotated[list, add_messages]

def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: list
) -> CompiledGraph:
    
    llm_with_tools = llm.bind_tools(tools=tools)

    preprocessor = RunnableLambda(lambda state: state["messages"])

    model_runnable = preprocessor | model

    # Augmented LLM with Tools Node function
    def tool_calling_llm(
        state: ChatAgentState,
        config: RunnableConfig):

        response = model_runnable.invoke(state, config)
        return {"messages": [response]}
    
    # Build the graph
    builder = StateGraph(ChatAgentState)
    builder.add_node("tool_calling_llm", RunnableLambda(tool_calling_llm))
    builder.add_node("tools", ChatAgentToolNode(tools=tools))
    builder.add_edge(START, "tool_calling_llm")

    # Conditional Edge
    builder.add_conditional_edges(
        "tool_calling_llm",
        # If the latest message (result) from LLM is a tool call -> tools_condition routes to tools
        # If the latest message (result) from LLM is a not a tool call -> tools_condition routes to END
        tools_condition,
        ["tools", END]
    )
    builder.add_edge("tools", "tool_calling_llm") # this is the key feedback loop
    builder.add_edge("tools", END)
    agent = builder.compile()

    return agent

class DocsAgent(ChatAgent):
    def __init__(self, config, tools):
        # Load config
        # When this agent is deployed to Model Serving, the configuration loaded here is replaced with the config passed to mlflow.pyfunc.log_model(model_config=...)
        self.config = ModelConfig(development_config=config)
        self.tools = tools
        self.agent = self._build_agent_from_config()

    def _build_agent_from_config(self):
        llm = ChatDatabricks(
            endpoint=self.config.get("endpoint_name"),
            temperature=self.config.get("temperature"),
            max_tokens=self.config.get("max_tokens"),
        )
        agent = create_tool_calling_agent(
            llm,
            tools=self.tools
        )
        return agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        # ChatAgent has a built-in helper method to help convert framework-specific messages, like langchain BaseMessage to a python dictionary
        request = {"messages": self._convert_messages_to_dict(messages)}

        output = self.agent.invoke(request)
        # Here 'output' is already a ChatAgentResponse, but to make the ChatAgent signature explicit for this demonstration we are returning a new instance
        return ChatAgentResponse(**output)
    

catalog = "agentic_ai"
schema = "databricks"

# TODO: Replace with your model serving endpoint
LLM_ENDPOINT = "databricks-meta-llama-3-3-70b-instruct"

baseline_config = {
    "endpoint_name": LLM_ENDPOINT,
    "temperature": 0.01,
    "max_tokens": 1000
}

LLM_ENDPOINT = "databricks-meta-llama-3-3-70b-instruct"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT)

uc_client = DatabricksFunctionClient()
set_uc_function_client(uc_client)



uc_tool_names = [f"{catalog}.{schema}.search_web"]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools=[*uc_toolkit.tools]

AGENT = DocsAgent(baseline_config, tools)

In [0]:
result = AGENT.predict({"messages": [{"role": "user", "content": "What is the latest news on OpenAI product releases? Provide the result in bullet points"}]})

In [0]:
from IPython.display import display, Markdown

display(Markdown(result.messages[-1].content))